# K-Means Clustering from Scratch
**Objective:** Implement K-Means Clustering (update rules, initialization sensitivity, K-Means++) from scratch using NumPy.

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# Set seed for reproducibility
np.random.seed(42)

## Problem Setup (Minimal)
**K-Means** aims to partition $N$ observations into $K$ clusters in which each observation belongs to the cluster with the nearest mean (centroid).

**Objective Function:** Minimize **Inertia** (Sum of Squared Errors - SSE):
$$ J = \sum_{j=1}^k \sum_{i \in C_j} ||x^{(i)} - \mu_j||^2 $$
where $\mu_j$ is the centroid of cluster $j$.

**Algorithm (Lloyd's):**
1.  Initialize $K$ centroids.
2.  **Assignment Step:** Assign each point to the nearest centroid.
3.  **Update Step:** Recompute centroids as the mean of assigned points.
4.  Repeat until convergence.

## Data

In [ ]:
# Generate Synthetic Data (Blobs)
n_samples = 300
n_features = 2
centers = 3
X, _ = make_blobs(n_samples=n_samples, centers=centers, cluster_std=1.0, random_state=42)

# Visualize
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], alpha=0.6, edgecolors='k')
plt.title("Synthetic Data for Clustering")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()

## Implementation (NumPy)

In [ ]:
def pairwise_distances(X, centroids):
    """
    Computes Euclidean distance squared between each point in X and each centroid.
    Returns matrix of shape (n_samples, k)
    """
    # Euclidean dist^2: ||a - b||^2 = ||a||^2 + ||b||^2 - 2<a, b>
    # Broadcasting approach for efficiency
    dist_sq = np.sum(X**2, axis=1).reshape(-1, 1) + np.sum(centroids**2, axis=1) - 2 * X.dot(centroids.T)
    return np.maximum(dist_sq, 0) # Safety against tiny negative numerical errors

def assign_clusters(distances):
    """Assigns nearest cluster index for each sample."""
    return np.argmin(distances, axis=1)

def compute_centroids(X, labels, k):
    """Recomputes centroids as mean of assigned points."""
    centroids = np.zeros((k, X.shape[1]))
    for i in range(k):
        points = X[labels == i]
        if len(points) > 0:
            centroids[i] = np.mean(points, axis=0)
        else:
            # Empty cluster handling: re-init with random point from X
            centroids[i] = X[np.random.randint(len(X))]
    return centroids

def compute_inertia(X, labels, centroids):
    """Computes Sum of Squared Errors."""
    inertia = 0
    for i in range(len(centroids)):
        points = X[labels == i]
        if len(points) > 0:
            inertia += np.sum((points - centroids[i])**2)
    return inertia

def init_centroids_random(X, k):
    """Randomly choose k samples as initial centroids."""
    indices = np.random.choice(X.shape[0], k, replace=False)
    return X[indices]

def init_centroids_kmeanspp(X, k):
    """K-Means++ Initialization."""
    centroids = []
    # 1. Choose first centroid randomly
    centroids.append(X[np.random.randint(X.shape[0])])
    
    for _ in range(1, k):
        # 2. Compute distances from points to nearest existing centroid
        dist_sq = np.array([min([np.inner(c-x, c-x) for c in centroids]) for x in X])
        
        # 3. Choose next centroid with probability proportional to dist^2
        probs = dist_sq / dist_sq.sum()
        cum_probs = np.cumsum(probs)
        r = np.random.rand()
        
        for i, p in enumerate(cum_probs):
            if r < p:
                centroids.append(X[i])
                break
                
    return np.array(centroids)

def fit_kmeans(X, k, max_iters=100, tol=1e-4, init="kmeans++"):
    if init == "kmeans++":
        centroids = init_centroids_kmeanspp(X, k)
    else:
        centroids = init_centroids_random(X, k)
        
    history = []
    
    for _ in range(max_iters):
        # Assignment
        dists = pairwise_distances(X, centroids)
        labels = assign_clusters(dists)
        
        # Calculate Inertia
        inertia = compute_inertia(X, labels, centroids)
        history.append(inertia)
        
        # Update Centroids
        new_centroids = compute_centroids(X, labels, k)
        
        # Check Convergence (Change in centroids)
        if np.allclose(centroids, new_centroids, atol=tol):
            break
            
        centroids = new_centroids
        
    return labels, centroids, history

## Experiments

In [ ]:
# Run Experiment A: Random Init
labels_rand, cent_rand, hist_rand = fit_kmeans(X, k=3, init="random")

# Run Experiment B: K-Means++ Init
labels_pp, cent_pp, hist_pp = fit_kmeans(X, k=3, init="kmeans++")

# Comparison
print(f"Random Init - Final Inertia: {hist_rand[-1]:.2f}, Iterations: {len(hist_rand)}")
print(f"K-Means++   - Final Inertia: {hist_pp[-1]:.2f}, Iterations: {len(hist_pp)}")

# Plot Convergence
plt.figure(figsize=(10, 5))
plt.plot(hist_rand, label='Random Init')
plt.plot(hist_pp, label='K-Means++')
plt.title("Convergence Speed: Inertia vs Iterations")
plt.xlabel("Iterations")
plt.ylabel("Inertia (SSE)")
plt.legend()
plt.grid(True)
plt.show()

## Visualization (Clusters)

In [ ]:
plt.figure(figsize=(14, 6))

# Random Init Plot
plt.subplot(1, 2, 1)
plt.scatter(X[:, 0], X[:, 1], c=labels_rand, cmap='viridis', alpha=0.5)
plt.scatter(cent_rand[:, 0], cent_rand[:, 1], c='red', marker='x', s=100, linewidths=3, label='Centroids')
plt.title(f"Random Init (Inertia: {hist_rand[-1]:.0f})")
plt.legend()

# K-Means++ Init Plot
plt.subplot(1, 2, 2)
plt.scatter(X[:, 0], X[:, 1], c=labels_pp, cmap='viridis', alpha=0.5)
plt.scatter(cent_pp[:, 0], cent_pp[:, 1], c='red', marker='x', s=100, linewidths=3, label='Centroids')
plt.title(f"K-Means++ (Inertia: {hist_pp[-1]:.0f})")
plt.legend()

plt.show()

## Choosing K (Elbow Method)

In [ ]:
k_values = range(1, 9)
inertias = []

for k in k_values:
    _, _, hist = fit_kmeans(X, k=k, init="kmeans++")
    inertias.append(hist[-1])
    
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertias, 'bo-', markersize=8)
plt.title("Elbow Method For Optimal K")
plt.xlabel("Number of clusters K")
plt.ylabel("Inertia")
plt.grid(True)
plt.show()

print("Observation: The 'elbow' appears around K=3, which matches our generated data.")

## Results & Takeaways
*   **K-Means++ Stability:** Random initialization can lead to poor local minima (bad clusters) or slower convergence. K-Means++ spreads out initial centroids, generally yielding better results faster.
*   **Number of Clusters (K):** The Elbow Method provides a visual heuristic to interpret data structure; adding more clusters gives diminishing returns on inertia reduction.
*   **Sensitivities:**
    *   **Scale:** K-Means assumes spherical clusters. If features have different scales (e.g., width vs height), standardization is required.
    *   **Shape:** Fails on non-convex shapes (like the 'moons' dataset from SVM) because it uses simple Euclidean distance.

## Next Steps
*   Explore **Dimensionality Reduction** to visualize high-dimensional clusters.
*   [Go to PCA Notebook](./pca-dimensionality-reduction.ipynb)